In [1]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [1]:
#from tensorflow.contrib.layers import xavier_initializer, flatten, fully_connected
import numpy as np
import keras
from keras.layers import Input, Dense, Conv1D, Conv2D, Conv2DTranspose, BatchNormalization
from keras.layers import LeakyReLU, PReLU, Reshape, Concatenate, Flatten
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
keras_backend = tf.keras.backend
keras_initializers = tf.keras.initializers
from data_ops import *
from file_ops import *
from models import *

import time
from tqdm import *
import h5py
import os,sys
import scipy.io.wavfile as wavfile

Using TensorFlow backend.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "C:\Users\skyun\miniconda3\envs\dl-env\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-fdba3484c83e>", line 9, in <module>
    keras_backend = tf.keras.backend
NameError: name 'tf' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\skyun\miniconda3\envs\dl-env\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no 

In [3]:
# Various GAN options
opts = {}
opts ['dirhead'] = "LSGAN"
opts ['z_off'] = True # set to True to omit the latent noise input
# normalization
#################################
# Only one of the follwoing should be set to True
opts ['applyinstancenorm'] = True
opts ['applybn'] = False
##################################
# Show model summary
opts ['show_summary'] = True

## Set the matfiles
clean_train_matfile = "./data/clean_train_segan1d.mat"
noisy_train_matfile = "./data/noisy_train_segan1d.mat"
noisy_test_matfile = "./data/noisy_test_segan1d.mat"

####################################################
# Other fixed options
opts ['window_length'] =  2**14
opts ['featdim'] = 1 # 1 since it is just 1d time samples
opts ['filterlength'] =  31
opts ['strides'] = 2
opts ['padding'] = 'SAME'
opts ['g_enc_numkernels'] = [16, 32, 32, 64, 64, 128, 128, 256, 256, 512, 1024]
opts ['d_fmaps'] = opts ['g_enc_numkernels'] # We use the same structure for discriminator
opts['leakyrelualpha'] = 0.3
opts ['batch_size'] = 100
opts ['applyprelu'] = True
opts ['preemph'] = 0.95
opts ['D_real_target'] = 1. # Use 0.9 0r 0.95 if you want to apply label smoothing

opts ['d_activation'] = 'leakyrelu'
g_enc_numkernels = opts ['g_enc_numkernels']
opts ['g_dec_numkernels'] = g_enc_numkernels[:-1][::-1] + [1]
opts ['gt_stride'] = 2
opts ['g_l1loss'] = 200.
opts ['d_lr'] = 0.0002
opts ['g_lr'] = 0.0002
opts ['random_seed'] = 111
    
n_epochs = 81
fs = 16000

# set flags for training or testing
TRAIN_SEGAN =  True
SAVE_MODEL =  True
LOAD_SAVED_MODEL = False
TEST_SEGAN =  True

modeldir = get_modeldirname(opts)
print ("The model directory is " + modeldir)
print ("_____________________________________")

if not os.path.exists(modeldir):
    os.makedirs(modeldir)

# Obtain the generator and the discriminator
D = discriminator(opts)
G = generator(opts)

# Define optimizers
g_opt = keras.optimizers.Adam(lr=opts['g_lr'])
d_opt = keras.optimizers.Adam(lr=opts['d_lr'])

# The G model has the wav and the noise inputs
wav_shape = (opts['window_length'], opts['featdim'])
z_dim1 = int(opts['window_length']/ (opts ['strides'] ** len(opts ['g_enc_numkernels'])))
z_dim2 = opts ['g_enc_numkernels'][-1]
wav_in_clean =  Input(shape=wav_shape, name="main_input_clean")
wav_in_noisy = Input(shape=wav_shape, name="main_input_noisy")
if not opts ['z_off']:
    z = Input (shape=(z_dim1, z_dim2), name="noise_input")
    G_wav = G([wav_in_noisy, z])
    G = Model([wav_in_noisy, z], G_wav)
else :
    G_wav = G(wav_in_noisy)
    G = Model(wav_in_noisy, G_wav)

d_out = D([wav_in_clean, wav_in_noisy])
D = Model([wav_in_clean, wav_in_noisy], d_out)
G.summary()
D.summary()

# compile individual models
D.compile(loss='mean_squared_error', optimizer=d_opt)
G.compile(loss='mean_absolute_error', optimizer=g_opt)

# for the combined model, we set the discriminator to be not trainable
D.trainable = False
D_out = D([G_wav, wav_in_noisy])
if not opts ['z_off']:
    G_D = Model(inputs=[wav_in_clean, wav_in_noisy, z], outputs=[D_out, G_wav])
else :
    G_D = Model(inputs=[wav_in_clean, wav_in_noisy], outputs=[D_out, G_wav])
G_D.summary()

G_D.compile(optimizer=g_opt,
            loss={'model_2': 'mean_absolute_error', 'model_4': 'mean_squared_error'},
            loss_weights = {'model_2' : opts['g_l1loss'], 'model_4': 1} )
print (G_D.metrics_names)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "C:\Users\skyun\miniconda3\envs\dl-env\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-d04eb30867f7>", line 52, in <module>
    modeldir = get_modeldirname(opts)
NameError: name 'get_modeldirname' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\skyun\miniconda3\envs\dl-env\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute 